3DBAG Data Preprocessing Script
=================================

The 3DBAG is an up-to-date data set containing 3D building models of the Netherlands. The 3DBAG is open data. It contains 3D models at multiple levels of detail, which are generated by combining two open data sets: the building data from the BAG and the height data from the AHN. The 3DBAG is updated regularly in order to remain up-to-date with the latest openly available building stock and elevation information.

**Data source:** https://3dbag.nl/en/download  
**Dataset size:** ~19 GB (ZIP), ~111 GB uncompressed


Overview

This script performs a complete preprocessing pipeline to **clip the nationwide 3DBAG dataset
to a smaller area of interest**, such as the city of **Utrecht**.


Processing Steps
1. Downloads the 3DBAG GeoPackage ZIP 
2. Inspects the zipped GeoPackage metadata (layers, CRS, geometry types, attributes)
3. Clips the building footprints layer (`pand`) to the Utrecht boundary
4. Writes the clipped data to a GeoPackage with a spatial index
5. Verifies the output by reading a small sample using GeoPandas


Key design choices:
- Uses GDAL Python bindings only 
- Reads the GeoPackage directly from ZIP using `/vsizip/`
- Avoids loading large datasets into memory
- Uses `/vsimem/` for temporary files (no disk clutter)

Requirements:
- Python 
- requests
- tqdm
- GDAL with Python bindings (osgeo)
- geopandas 

In [1]:
# -----------------------------
# Imports
# -----------------------------

from pathlib import Path
import requests
from osgeo import gdal, ogr, osr
import time
from tqdm import tqdm



In [2]:

# -----------------------------
# Static configuration
# -----------------------------

DATA_URL = "https://data.3dbag.nl/v20250903/3dbag_nl.gpkg.zip"
ZIP_FILE = "static/data/3dbag_nl.gpkg.zip"

BOUNDARY_FILE = "static/data/utrecht.geojson"
BOUNDARY_REPROJECTED = "static/data/utrecht_7415.geojson"
SOURCE_LAYER = "pand"

OUTPUT_GPKG = "static/data/utrecht_pand_clip.gpkg"
OUTPUT_LAYER = "pand_utrecht"

TARGET_EPSG = 7415  # Amersfoort / RD New + NAP height

In [3]:
# -----------------------------
# Download dataset (streaming)
# -----------------------------

def download_file(url: str, out_path: Path):
    if out_path.exists():
        print(f"✔ ZIP already exists: {out_path}")
        return

    print(f"⬇ Downloading:\n  {url}")

    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        total = int(r.headers.get("Content-Length", 0))

        with open(out_path, "wb") as f, tqdm(
            total=total,
            unit="B",
            unit_scale=True,
            unit_divisor=1024,
            desc="Downloading",
        ) as bar:
            for chunk in r.iter_content(chunk_size=8 * 1024 * 1024):  # 8 MB
                if chunk:
                    f.write(chunk)
                    bar.update(len(chunk))

    print("✔ Download complete")


# Call
download_file(DATA_URL, Path(ZIP_FILE))

✔ ZIP already exists: static/data/3dbag_nl.gpkg.zip


In [4]:
# -----------------------------
# Inspect zipped GeoPackage
# -----------------------------

VSIZIP_PATH = f"/vsizip/{ZIP_FILE}"

def inspect_zipped_gpkg(vsizip_path: str):
    gdal.UseExceptions()
    ogr.UseExceptions()

    ds = gdal.OpenEx(vsizip_path, gdal.OF_VECTOR)
    if ds is None:
        raise RuntimeError(f"Could not open dataset: {vsizip_path}")

    print("\n==============================")
    print("DATASET INFORMATION")
    print("==============================")
    print("Path   :", vsizip_path)
    print("Driver :", ds.GetDriver().GetName())
    print("Layers :", ds.GetLayerCount())

    for i in range(ds.GetLayerCount()):
        layer = ds.GetLayerByIndex(i)
        defn = layer.GetLayerDefn()

        print("\n------------------------------")
        print(f"Layer {i + 1}: {layer.GetName()}")

        # Geometry
        geom_type = ogr.GeometryTypeToName(defn.GetGeomType())
        print("Geometry type :", geom_type)

        # Feature count
        print("Feature count :", layer.GetFeatureCount())

        # Extent
        extent = layer.GetExtent()
        if extent:
            print(f"Extent        : ({extent[0]:.3f}, {extent[2]:.3f}) - ({extent[1]:.3f}, {extent[3]:.3f})")

        # CRS
        srs = layer.GetSpatialRef()
        if srs:
            epsg = srs.GetAuthorityCode(None)
            if epsg:
                print("CRS           : EPSG:" + epsg)
            else:
                print("CRS           : Custom / Compound CRS")
        else:
            print("CRS           : None")

        # Geometry column
        print("Geometry col  :", layer.GetGeometryColumn() or "geom")

        # Attributes
        print("\nAttributes:")
        for j in range(defn.GetFieldCount()):
            field = defn.GetFieldDefn(j)
            print(
                f"  - {field.GetName()} "
                f"({field.GetFieldTypeName(field.GetType())})"
            )

    ds = None
    print("\n✅ Inspection complete.")


# Run
inspect_zipped_gpkg(VSIZIP_PATH)


DATASET INFORMATION
Path   : /vsizip/static/data/3dbag_nl.gpkg.zip
Driver : GPKG
Layers : 7

------------------------------
Layer 1: lod12_3d
Geometry type : 3D Multi Polygon
Feature count : 10783975
Extent        : (13603.331, 306900.406) - (277924.312, 612658.062)
CRS           : EPSG:7415
Geometry col  : geom

Attributes:
  - identificatie (String)
  - b3_pand_deel_id (Integer64)
  - labels (String)

------------------------------
Layer 2: lod12_2d
Geometry type : Polygon
Feature count : 10783944
Extent        : (13603.331, 306900.406) - (277924.312, 612658.062)
CRS           : EPSG:7415
Geometry col  : geom

Attributes:
  - b3_h_50p (Real)
  - b3_h_70p (Real)
  - b3_h_max (Real)
  - b3_h_min (Real)
  - b3_dd_id (Integer64)
  - identificatie (String)
  - b3_pand_deel_id (Integer64)

------------------------------
Layer 3: lod13_3d
Geometry type : 3D Multi Polygon
Feature count : 10783975
Extent        : (13603.331, 306900.406) - (277924.312, 612658.062)
CRS           : EPSG:7415
Ge

In [5]:

# -----------------------------
# Reproject boundary
# -----------------------------
def reproject_boundary(boundary_path: str, target_epsg: int) -> str:
    src = gdal.OpenEx(boundary_path, gdal.OF_VECTOR)

    srs = osr.SpatialReference()
    srs.ImportFromEPSG(target_epsg)


    gdal.VectorTranslate(
        BOUNDARY_REPROJECTED,
        src,
        options=gdal.VectorTranslateOptions(
            format="GeoJSON",
            dstSRS=srs.ExportToWkt(),
        )
    )

    src = None
    return BOUNDARY_REPROJECTED


# -----------------------------
# Clip pand layer
# -----------------------------
def clip_pand_layer(zip_path: Path, cutline_path: str):

    vsizip = f"/vsizip/{zip_path}"

    # Remove existing output
    if Path(OUTPUT_GPKG).exists():
        Path(OUTPUT_GPKG).unlink()

    gdal.VectorTranslate(
        OUTPUT_GPKG,
        vsizip,
        options=gdal.VectorTranslateOptions(
            format="GPKG",
            layers=[SOURCE_LAYER],
            options=[
                "-clipsrc", cutline_path,     # <-- IMPORTANT FIX
                "-nln", OUTPUT_LAYER,
            ],
            layerCreationOptions=[
                "SPATIAL_INDEX=YES"
            ],
        )
    )

    


In [7]:

# -----------------------------
# Main pipeline
# -----------------------------

def main():
    gdal.UseExceptions()
    ogr.UseExceptions()

    #  Reproject Utrecht boundary
    print("Reprojecting Utrecht boundary to EPSG:7415")
    reproject_boundary(BOUNDARY_FILE, TARGET_EPSG)

    # Clip buildings (pand)
    print("Clipping pand layer to Utrecht")
    clip_pand_layer(Path(ZIP_FILE), BOUNDARY_REPROJECTED)

    print(f"✅ Done! Output written to: {OUTPUT_GPKG}")


if __name__ == "__main__":
    main()

🔄 Reprojecting Utrecht boundary to EPSG:7415
✂ Clipping pand layer to Utrecht
✅ Done! Output written to: static/data/utrecht_pand_clip.gpkg


Warning 1: Layer 'pand_utrecht' has been declared with non-Z geometry type Polygon, but it does contain geometries with Z. Setting the Z=2 hint into gpkg_geometry_columns
Warning 1: A geometry of type GEOMETRYCOLLECTION is inserted into layer pand_utrecht of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
ERROR 1: TopologyException: side location conflict at 141587.64101890757 440071.65598739497 0. This can occur if the input geometry is invalid.
ERROR 1: TopologyException: side location conflict at 129356.60816622103 454707.70726659981 0. This can occur if the input geometry is invalid.
ERROR 1: TopologyException: side location conflict at 115931.24622404104 468505.08130273438 0. This can occur if the i

In [8]:

# Read only first few rows to confirm output

import geopandas as gpd

gdf = gpd.read_file(OUTPUT_GPKG, layer=OUTPUT_LAYER, rows=5)

print("Columns:")
print(list(gdf.columns))

print("\nHead (attributes only):")
print(gdf.drop(columns="geometry").head())


Columns:
['b3_bag_bag_overlap', 'b3_bouwlagen', 'b3_dak_type', 'b3_extrusie', 'b3_h_maaiveld', 'b3_h_nok', 'b3_is_glas_dak', 'b3_kas_warenhuis', 'b3_kwaliteitsindicator', 'b3_mutatie_ahn3_ahn4', 'b3_mutatie_ahn4_ahn5', 'b3_n_nok', 'b3_n_vlakken', 'b3_nodata_fractie_ahn3', 'b3_nodata_fractie_ahn4', 'b3_nodata_fractie_ahn5', 'b3_nodata_radius_ahn3', 'b3_nodata_radius_ahn4', 'b3_nodata_radius_ahn5', 'b3_opp_buitenmuur', 'b3_opp_dak_plat', 'b3_opp_dak_schuin', 'b3_opp_grond', 'b3_opp_scheidingsmuur', 'b3_puntdichtheid_ahn3', 'b3_puntdichtheid_ahn4', 'b3_puntdichtheid_ahn5', 'b3_pw_bron', 'b3_pw_datum', 'b3_pw_onvoldoende', 'b3_pw_selectie_reden', 'b3_rmse_lod12', 'b3_rmse_lod13', 'b3_rmse_lod22', 'b3_t_run', 'b3_val3dity_lod12', 'b3_val3dity_lod13', 'b3_val3dity_lod22', 'b3_volume_lod12', 'b3_volume_lod13', 'b3_volume_lod22', 'begingeldigheid', 'documentdatum', 'documentnummer', 'eindgeldigheid', 'eindregistratie', 'geconstateerd', 'identificatie', 'oorspronkelijkbouwjaar', 'status', 'tijd